In [1]:
import csv 
import requests 
import xml.etree.ElementTree as ET 
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.text import Text

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
temp = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Necessary Codes/pklfile/data.pkl")
temp.shape

(2153, 3)

In [4]:
((temp.iloc[1])['Full'])

' civil appeal no. 350 of 1970. appeal by special leave from the judgment and order dated march 21, 1966 of the madras high court in o.s. appeal no. 11 of 1962. b. r. l. lyengar, m. v. goswami, s. r. agarwala, a. t. m. sampat and e. c. agrawala, for the appellant. u. p. singh, santok singh, ugra shankar prasad and shiva pujan singh, for respondent no. 1. s. p. sinha and m. i. khowaja, for respondents nos. 2 and 3. the judgment of the court was delivered by ray, j. this appeal is by special leave from the judgment dated 21 march, 1966 of the madras high court dismissing the appeal preferred by the appellant against the decree holders\' application for execution of the decree. the appellant is one of the judgment-debtors brought-on record as legal representative of a deceased judgment debtor lala baijnath prasad. respondent no. 1 lakshman prasad gupta was one of the plaintiffs. pratap chand and basudeb prasad respondents nos. 2 and 3 respectively are the sons of a judgment-debtor girdhar

In [5]:
temp["col_len"] = temp.apply(lambda row:len(row.Full),axis = 1,result_type = "expand")
temp.drop(["Doc"],axis = 1,inplace = True)

In [6]:
temp

,Summary,Full,col_len
0,the appellant terminated the service of a wor...,civil appeal no. 1914 of 1968. appeal by spec...,31013
1,"the first respondent, in 1938, obtained a dec...",civil appeal no. 350 of 1970. appeal by speci...,20014
2,the assessee was a private limited company. i...,civil appeals nos. 2380 and 2381 of 1966. app...,13207
3,"notices under s. 22(2) of the income-tax act,...",civil appeals nos. 23 5 and 236 of 1967. appe...,8822
4,d. b and j were partners in a firm which carr...,civil appeals nos. 1277 to 1279 and 1280 to 1...,13558
...,...,...,...
2148,arishtams and asavas are ayurvedic preparatio...,civil appeal no. 1868 of 1974. from the judgm...,10498
2149,the appellant was an assessee who derived inc...,civil appeal nos. 80 & 81 of 1975. from the j...,7540
2150,"sub-s. (1) of s. 25 of the wealth tax act, 19...",civil appeal nos. 88 to 93 to 1974. from the ...,9091
2151,appellants nos. 1 and 2 were elected on june ...,civil appeal no. 1994 of 1989. from the judgm...,33810


In [7]:
## pip install spacy
## other prerequisites to be added
import spacy
from spacy.lang.en import English
nlp = English()
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 21787990
# nlp.add_pipe(nlp.create_pipe('sentencizer'))

nltk.download('stopwords')
nltk.download('punkt')
is_remove_stopwords = True
if is_remove_stopwords:
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
ENTITY_ENUM = {
    '': '',
    'PERSON': 'person',
    'NORP': 'nationality',
    'FAC': 'facility',
    'ORG': 'organization',
    'GPE': 'country',
    'LOC': 'location',
    'PRODUCT': 'product',
    'EVENT': 'event',
    'WORK_OF_ART': 'artwork',
    'LANGUAGE': 'language',
    'DATE': 'date',
    'TIME': 'time',
    'PERCENT': 'number',
    'MONEY': 'number',
    'QUANTITY': 'number',
    'ORDINAL': 'number',
    'CARDINAL': 'number',
    'LAW': 'law'
}

NUMERIC_TYPES = set([
    'DATE',
    'TIME',
    'PERCENT',
    'MONEY',
    'QUANTITY',
    'ORDINAL',
    'CARDINAL',
])

In [9]:
import re

def clean_string(text):
    text = str(text)
  # Replace weird chars in text
    text = re.sub("’", "'", text) # special single quote
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    text = re.sub("？", "?", text) 
    text = re.sub("…", " ", text) 
    text = re.sub("é", "e", text)
  
    # Clean shorthands 
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub(r"(\W|^)([0-9]+)[kK](\W|$)", r"\1\g<2>000\3", text) # better regex provided by @armamut
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
    # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"
    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)

    # indian rupees
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)

    # the single 's' in this stage is 99% of not clean text, just kill it
    text = re.sub(' s ', " ", text)
    
    # reduce extra spaces into single spaces
    text = re.sub('[\s]+', " ", text)
    text = text.strip()

    return text


In [10]:
def vote_dictionary_generation(doc,vote_dict):

  # construct vote dictionary
  for token in doc:
    if token.lower_ not in vote_dict:
      vote_dict[token.lower_] = {}
    if token.ent_type_ not in vote_dict[token.lower_]:
      vote_dict[token.lower_][token.ent_type_] = 0
      # if the token has_vector is True, maybe we shouldn't record its 
      vote_dict[token.lower_][token.ent_type_] += 1 # TODO: not sure if storing in lowercase form is safe ?

def entity_lookup_generation(vote_dict,word_ent_type_dict,word_ent_type_second_dict):
  threshold_of_second_type = 3
  # vote for what should the type be   
  for key in vote_dict:
    # non-type has lower priority
    if '' in vote_dict[key]:
      vote_dict[key][''] = vote_dict[key][''] - 0.1

    ents = list(vote_dict[key].keys())
    bi_list = [
              ents,
              [vote_dict[key][ent] for ent in ents]
              ]
    # if several ent_type_ have same count, just let it go, making them share same ent_type_ is enough
    # TODO: if have time, can design a better metric to deal with second graded type
    sorted_idx = np.argsort(bi_list[1])
    if sorted_idx.shape[0]>1:
      best_idx = sorted_idx[-1]
      second_idx = sorted_idx[-2]
      word_ent_type_dict[key] = bi_list[0][best_idx]
      if bi_list[1][second_idx]>threshold_of_second_type:
        word_ent_type_second_dict[key] = bi_list[0][second_idx]
    else:
      best_idx = sorted_idx[-1]
      word_ent_type_dict[key] = bi_list[0][best_idx]

In [11]:
## This function is used in our case - Checks entity type of a token
def token_type_lookup(token, report_detail=False):
    
    if type(token)==str:
        token = nlp(token)[0]
        
    key = token.lower_
    
    try:
        if report_detail:
            print(ENTITY_ENUM[word_ent_type_dict[key]], ' <= ', {ENTITY_ENUM[ent_t] : vote_dict[key][ent_t] for ent_t in vote_dict[key]} )

        return word_ent_type_dict[key]
    
    except KeyError:
        return ''

def is_token_has_second_type(token):
    
    if type(token)==str:
        token = nlp(token)[0]
        
    key = token.lower_
    
    try:
        return key in word_ent_type_second_dict
    except KeyError:
        return False

def token_second_type_lookup(token, report_detail=False):
    
    if type(token)==str:
        token = nlp(token)[0]
        
    key = token.lower_
    
    try:
        if report_detail:
            print(ENTITY_ENUM[word_ent_type_second_dict[key]], ' <= ', {ENTITY_ENUM[ent_t] : vote_dict[key][ent_t] for ent_t in vote_dict[key]} )

        return word_ent_type_second_dict[key]
    except KeyError:
        return ''

In [12]:
exception_list =  set(['need']) # spaCy identifies need's lamma as 'ne', which is not we want
numeric_types = set(['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL'])

def process_text_with_spacy(spacy_obj, debug=False, show_fail=False, idx=None):

    def not_alpha_or_digit(token):
        ch = token.text[0]
        return not (ch.isalpha() or ch.isdigit())
    
    result_word_list = []
    res = ''
    
    # for continuous entity type string, we need only single term. 
    # EX: "2017-01-01"
    # => "time time time" (X)
    # => "time" (O)
    previous_ent_type = None
    
    is_a_word_parsed_fail = False
    fail_words = []
    
    for token in spacy_obj:
        
        global_ent_type = token_type_lookup(token)
        
        # problematic token, use its base form
        if token.text in exception_list:
            
            previous_ent_type = None
            result_word_list.append(token.text)
            
        # skip none words tokens
        elif not_alpha_or_digit(token) or token.text==' ' or token.text=='s':
            previous_ent_type = None
            if debug: print(token.text, ' : remove punc or special chars')
            
            
        # if the "remove stop word" flag is set to True
        elif is_remove_stopwords and token.lemma_ in stopwords:
            previous_ent_type = None
            if debug: print(token.text, ' : remove stop word')
        
        
        # contiguous same type, skip it
        elif global_ent_type==previous_ent_type or token.ent_type_==previous_ent_type:
            if debug: print('contiguous same type')
        elif global_ent_type in NUMERIC_TYPES and previous_ent_type in NUMERIC_TYPES:
            if debug: print('contiguous numeric')
        elif token.ent_type_ in NUMERIC_TYPES and previous_ent_type in NUMERIC_TYPES:
            if debug: print('contiguous numeric')
                
        
        # number without an ent_type_
        elif token.text.isdigit():
            
            if debug: print(token.text, 'force to be number')
                
            if previous_ent_type in NUMERIC_TYPES:
                pass
            else:
                previous_ent_type = 'CARDINAL' # any number type would be okay
                result_word_list.append('number')

    
        # replace proper nouns into name entities. 
        # EX:
        # Original : Taiwan is next to China
        # Result   : country is next to country 
        elif global_ent_type!='':
            
            result_word_list.append(ENTITY_ENUM[global_ent_type])
            previous_ent_type = global_ent_type
            if debug: print(token.text, ' : sub ent_type:', ENTITY_ENUM[global_ent_type])
            
            
        # Identify if a word is proper noun or not, if it is a proper noun, we'll try to use second highest rated ent_type_
        #
        # A proper noun has following special patterns:
        #     1. its lemma_ (base form) returned by spaCy is just its lowercase form
        #     2. if one of its character except the first character is uppercase, it is a propernoun (in most cases)
        # except the special cases like "I LOVE YOU", we cal say that if (1.) and (2.), then the token is proper noun
        #
        # for cases like "Tensorflow", we have no good rule to identify it is a proper noun or not ... let's just move on
        elif token.lower_==token.lemma_ and token.text[1:]!=token.lemma_[1:] and is_token_has_second_type(token):
            second_type = token_second_type_lookup(token)
            result_word_list.append(ENTITY_ENUM[second_type])
            if debug: print(token.text, ' : use second ent_type:', ENTITY_ENUM[second_type])
            previous_ent_type = second_type
        
        
        # words arrive here are either "extremely common" or "extremely rare and has no method to deal with"
        else:
            # A weird behavior of SpaCy, it substitutes [I, my, they] into '-PRON-'
            if token.lemma_=='-PRON-':
                result_word_list.append(token.lower_)
                res = token.lower
                previous_ent_type = None
            
            # the lemma can be identified by GloVe
            elif nlp(token.lemma_)[0].has_vector:
                result_word_list.append(token.lemma_)
                res = token.lemma_
                previous_ent_type = None
            
            # the lemma cannot be identified, very probably a proper noun
            elif is_token_has_second_type(token):
                second_type = token_second_type_lookup(token)
                result_word_list.append(ENTITY_ENUM[second_type])
                res = ENTITY_ENUM[second_type]
                previous_ent_type = second_type
                if debug: print(token.text, ' : use second ent_type in else :', ENTITY_ENUM[second_type])
            
            # the lemma is not in glove and Spacy can't identify if it is a proper noun, last try, 
            #      if the word itself can be identified by GloVe or not
            elif nlp(token.lower_)[0].has_vector:
                result_word_list.append(token.lower_)
                res = token.lower_
                previous_ent_type = None
                if debug: print(token.text, ' : the token itself can be identified :', token.lower_)
            elif token.has_vector:
                result_word_list.append(token.text)
                res = token.text
                previous_ent_type = None
                if debug: print(token.text, ' : the token itself can be identified :', token.text)
                
            # Damn, I have totally no idea what's going on
            # You got to deal with it by yourself
            # In my case, I use fasttext to deal with it
            else:
                is_a_word_parsed_fail = True
                fail_words.append(token.text)
                previous_ent_type = None
                
                if debug: print(token.text, ' : can\'t identify, replace with "something"')
                
    
    if show_fail and is_a_word_parsed_fail:
        if idx!=None:
            print('At qid=', idx)
        print('Fail words: ', fail_words)
        print('Before:', spacy_obj.text)
        print('After: ', ' '.join(result_word_list))
        print('====================================================================')
    
    return np.array(result_word_list)

In [13]:
len(temp)

2153

In [14]:
## Generates vote dictionary which is used for generating Entity dictionary
import json
import time

# STEP 2: fill batch number according to your convinience

batch_size = 30

# STEP 3: batch wise processing
# batch no to be filled inside the range value
# tabulate the batch number according to your allocation of document to be processed
for i in range(0, 10):
  vote_dict = {}
  word_ent_type_dict = {}
  word_ent_type_second_dict = {} 
  max_length = 0

  start = i * batch_size
  end = start + batch_size - 1
  start_time = time.time()
  print("Start Target: ", start)
  print("End Target  : ", end)

  for i,row in temp.iterrows():
    if(i>=start and i <= end):
      print("vote dict, doc no: ", i, "of length: ", row['col_len'])
      for col in temp.columns:
        doc = nlp(clean_string(row[col]).lower())
        vote_dictionary_generation(doc, vote_dict)
  
  entity_lookup_generation(vote_dict, word_ent_type_dict, word_ent_type_second_dict)

  # STEP 4: change the path accordingly
  # would suggest not to change the naming convention
  with open('/content/drive/MyDrive/Colab Notebooks/Necessary Codes/newdictionary/word_ent_type_dict_' + str(start) + '_to_' + str(end) +'.json', 'w') as fp1:
      json.dump(word_ent_type_dict, fp1)
  
  end_time = time.time()
  print("\nword_ent_type_dict Write Successful in time : ", (end_time - start_time)/60 , " min")

  # try:
  df = pd.DataFrame()

  start_time = time.time()
  for i,row in temp.iterrows():
    summary = []
    full = []
    ori_full = []

    # For Summary
    if(i>=start and i <= end):
        print("summary, doc no: ", i, "of length: ", len((row["Summary"])[:-110]))
        for sent in sent_tokenize(clean_string((row["Summary"])[:-110]).lower()): 
            doc = nlp(sent)
            s = process_text_with_spacy(doc) # Advanced Text Cleaning
            res = " ".join(s)
            summary.append(res)
            
    # For Full text
    if(i>=start and i <= end):
        print("full, doc no   : ", i, "of length: ", len((row["Full"])[56:-468]))
        for sent in sent_tokenize(clean_string((row["Full"])[56:-468]).lower()):
            doc = nlp(sent)
            s = process_text_with_spacy(doc)
            res = " ".join(s)
            full.append(res)
            ori_full.append(sent)
      
    sum_join = "*".join(summary)
    full_join = "*".join(full)
    ori_full_join = "*".join(ori_full)

    df_obj = pd.DataFrame({"summary":[sum_join], "full":[full_join], "full_orignal":[ori_full_join], "Id":[i]})
    df = pd.concat([df,df_obj])

  # STEP 5: change path to the destination folder accordingly
  # would suggest not to change the naming convention
  (df.iloc[start:end+1]).to_pickle('/content/drive/MyDrive/Colab Notebooks/Necessary Codes/newdataframe/dataframe_processed_' + str(start) + '_to_' + str(end) +'.pickle')

  end_time = time.time()
  print("\nDataframe Write to Disk Successful in time: ", (end_time - start_time)/60 , " min")
print(vote_dict)

Start Target:  0
End Target  :  29
vote dict, doc no:  0 of length:  31013
vote dict, doc no:  1 of length:  20014
vote dict, doc no:  2 of length:  13207
vote dict, doc no:  3 of length:  8822
vote dict, doc no:  4 of length:  13558
vote dict, doc no:  5 of length:  8678
vote dict, doc no:  6 of length:  10728
vote dict, doc no:  7 of length:  15363
vote dict, doc no:  8 of length:  4407
vote dict, doc no:  9 of length:  10495
vote dict, doc no:  10 of length:  15644
vote dict, doc no:  11 of length:  11831
vote dict, doc no:  12 of length:  9815
vote dict, doc no:  13 of length:  6063
vote dict, doc no:  14 of length:  21091
vote dict, doc no:  15 of length:  25290
vote dict, doc no:  16 of length:  4230
vote dict, doc no:  17 of length:  28983
vote dict, doc no:  18 of length:  13669
vote dict, doc no:  19 of length:  19772
vote dict, doc no:  20 of length:  12959
vote dict, doc no:  21 of length:  18495
vote dict, doc no:  22 of length:  28274
vote dict, doc no:  23 of length:  880

In [15]:
vote_dict


{'the': {'': 0.9,
  'ORG': 1,
  'DATE': 1,
  'LAW': 1,
  'LOC': 1,
  'PERSON': 1,
  'TIME': 1,
  'FAC': 1},
 'settlement': {'': 0.9},
 'officer': {'': 0.9},
 'under': {'': 0.9},
 'madras': {'': 0.9, 'PERSON': 1, 'ORG': 1, 'LOC': 1, 'FAC': 1},
 'estates': {'': 0.9, 'ORG': 1},
 '(': {'': 0.9,
  'LAW': 1,
  'PERSON': 1,
  'ORG': 1,
  'DATE': 1,
  'WORK_OF_ART': 1,
  'PERCENT': 1},
 'abolition': {'': 0.9, 'ORG': 1},
 'and': {'': 0.9, 'DATE': 1, 'CARDINAL': 1, 'PERSON': 1, 'ORG': 1, 'LAW': 1},
 'conversion': {'': 0.9},
 'into': {'': 0.9},
 'ryotwari': {'': 0.9},
 ')': {'': 0.9,
  'CARDINAL': 1,
  'PERSON': 1,
  'MONEY': 1,
  'DATE': 1,
  'LAW': 1,
  'ORG': 1,
  'WORK_OF_ART': 1,
  'PERCENT': 1},
 'act': {'': 0.9, 'ORG': 1, 'LAW': 1, 'LOC': 1, 'DATE': 1},
 ',': {'': 0.9,
  'DATE': 1,
  'GPE': 1,
  'TIME': 1,
  'ORG': 1,
  'MONEY': 1,
  'PERSON': 1,
  'LAW': 1},
 '1948': {'DATE': 1, '': 0.9, 'CARDINAL': 1},
 'suo': {'PERSON': 1},
 'motu': {'PERSON': 1},
 'made': {'': 0.9},
 'an': {'': 0.9, 'T

In [20]:
try:
  df = pd.DataFrame()
  for i,row in temp.iterrows():
    if i % 500 == 0:
      print(i)
    summary = []
    full = []
    # For Summary
    for sent in sent_tokenize(clean_string(row["Summary"]).lower()): # Clean_string - Basic Text Cleaning
        summary.append(sent)
    # For Full text
    for sent in sent_tokenize(clean_string(row["Full"]).lower()):
        full.append(sent)
    
    sum_join = "*".join(summary)
    full_join = "*".join(full)

    df_obj = pd.DataFrame({"Summary":[sum_join],"Full":[full_join],"Id":[i]})
    # This data frame consists of Text whose format is like raw original dataframe ( summary and full ) 
    # but is processed to clean format
    df = pd.concat([df,df_obj])
except Exception as e:
  print("saving df of length",len(df))
  print(e)

0
500
1000
1500
2000


In [21]:
print(df)


                                              Summary  \
0   the appellant terminated the service of a work...   
0   the first respondent, in 1938, obtained a decr...   
0   the assessee was a private limited company.*it...   
0   notices under s. 22(2) of the income-tax act, ...   
0   d. b and j were partners in a firm which carri...   
..                                                ...   
0   arishtams and asavas are ayurvedic preparation...   
0   the appellant was an assessee who derived inco...   
0   sub-s. (1) of s. 25 of the wealth tax act, 195...   
0   appellants nos.*1 and 2 were elected on june 3...   
0   the defence of india act, 1962 was passed by p...   

                                                 Full    Id  
0   civil appeal no.*1914 of 1968. appeal by speci...     0  
0   civil appeal no.*350 of 1970. appeal by specia...     1  
0   civil appeals nos.*2380 and 2381 of 1966. appe...     2  
0   civil appeals nos.*23 5 and 236 of 1967. appea...     3  
0   c

In [22]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Necessary Codes/data.csv")